In [1]:
import os
import dotenv

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

In [2]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.core import Settings
import os

llm = OpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
)

# use another embedding model here
# change this code
embed_model = OpenAIEmbedding(
    model="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
)

Settings.embed_model = embed_model

In [3]:
import phoenix as px
px.launch_app()

/home/vscode/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [4]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [5]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="../local_index_large_embedding" # you will also have to change this
    )
    index = load_index_from_storage(storage_context)
except:
    #Note: we have to reduce the batch size to stay within the token limits of the free service
    documents = SimpleDirectoryReader("../Big Star Collectibles").load_data()
    index = VectorStoreIndex.from_documents(documents, insert_batch_size=150)
    index.storage_context.persist(persist_dir="../local_index_large_embedding")

In [6]:
# solution
query_engine = index.as_query_engine(
  llm=llm
)
query_engine.query("When did the story for Big Star Collectibles Start?")

Response(response='The story for Big Star Collectibles began in 2013 at the International Arts Conference.', source_nodes=[NodeWithScore(node=TextNode(id_='9d7a43b4-2f15-4827-aed4-c505da8aeeb3', embedding=None, metadata={'file_path': '/workspaces/introduction-to-retrieval-augmented-generation-rag-with-hands-on-projects-2023442/chapter_2/../Big Star Collectibles/Our Story.txt', 'file_name': 'Our Story.txt', 'file_type': 'text/plain', 'file_size': 877, 'creation_date': '2025-04-21', 'last_modified_date': '2025-04-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1a3a0d2c-6daf-4bc1-8d96-d7a4ac8413c9', node_type='4', metadata={'file_path': '/workspaces/introduction-to-retrieval-augmented-generation-rag-with

In [7]:
query_engine.query("Who started Big Star Collectibles?")

Response(response='Big Star Collectibles was started by Saura Chen.', source_nodes=[NodeWithScore(node=TextNode(id_='9d7a43b4-2f15-4827-aed4-c505da8aeeb3', embedding=None, metadata={'file_path': '/workspaces/introduction-to-retrieval-augmented-generation-rag-with-hands-on-projects-2023442/chapter_2/../Big Star Collectibles/Our Story.txt', 'file_name': 'Our Story.txt', 'file_type': 'text/plain', 'file_size': 877, 'creation_date': '2025-04-21', 'last_modified_date': '2025-04-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1a3a0d2c-6daf-4bc1-8d96-d7a4ac8413c9', node_type='4', metadata={'file_path': '/workspaces/introduction-to-retrieval-augmented-generation-rag-with-hands-on-projects-2023442/chapter_2/..